Aluno: Luiz Fernando de Oliveira Pereira

RA: 267356

In [1]:
# Ativando o multithreading no Google Colab

using Base.Threads
println("Núcleos disponíveis = ", nthreads())


Núcleos disponíveis = 2


Simulando um cálculo pesado para o loop:

Calculando a soma dos logaritmos de números aleatórios 80 milhões de vezes.

No código abaixo o que está acontecendo:

Gera um número aleatório x

Calcula log(x)

Soma no acumulador s

Repete N vezes

Sem Paralelismo

In [2]:
using Random
using Base.Threads

function calc_serie(N)
    s = 0.0
    for i in 1:N
        x = rand()
        s += log(x)
    end
    return s
end

N = 80_000_00  # 80 milhões
println("Executando loop em série...")
@time r1 = calc_serie(N)


Executando loop em série...
  0.216661 seconds (32.51 k allocations: 1.537 MiB, 10.32% compilation time)


-8.002411990166867e6

Com Paralelismo, usamos:

`Threads.@threads ` para dividir as iterações

Um vetor “thread-safe” para armazenar resultados parciais

In [3]:
function calc_paralelo(N)
    n = nthreads()
    partial = zeros(Float64, n)

    @threads for i in 1:N
        id = threadid()
        x = rand()
        partial[id] += log(x)
    end

    return sum(partial)
end

println("Executando loop em paralelo com $(nthreads()) threads...")
@time r2 = calc_paralelo(N)


Executando loop em paralelo com 2 threads...
  0.278455 seconds (130.30 k allocations: 6.576 MiB, 76.53% compilation time)


-8.00023740617175e6

| Execução            | Tempo (s)  | Alocações      | Memória usada | Resultado numérico |
|--------------------|------------|----------------|----------------|--------------------|
| Sem Paralelismo     | 0.216661 s | 32.51 mil       | 1.537 MiB      | -8.002411990166867e6 |
| Com Paralelismo (2 threads) | 0.278455 s | 130.30 mil      | 6.576 MiB      | -8.00023740617175e6 |


Nesse exemplo acima, percebemos que o loop sem paralelismo foi mais rápido

Alguns dos motivos de no paralelo o tempo ter aumento foi porque:

Há overhead de criação e coordenação de threads

Alocou 4× mais memória

Houve mais compilação na primeira execução

Isso é normal se a tarefa não é suficientemente pesada para se beneficiar do paralelismo.

Agora um exemplo onde o loop em paralelo é mais compensador

No exemplo abaixo temos o cálculo pesado de raízes + logaritmos + seno milhões de vezes com N = 50 milhões

In [12]:
using Base.Threads
using Random
using Plots

println("Threads ativos = ", nthreads())


# Função PESADA (ideal para paralelismo)

function trabalho_pesado(x)
    return sqrt(x) * sin(x) * log(x + 1)
end

# Loop em SÉRIE

function loop_serie(N)
    s = 0.0
    for i in 1:N
        x = rand()
        s += trabalho_pesado(x)
    end
    return s
end


# Loop em PARALELO

function loop_paralelo(N)
    partial = zeros(Float64, nthreads())

    @threads for i in 1:N
        tid = threadid()
        x = rand()
        partial[tid] += trabalho_pesado(x)
    end

    return sum(partial)
end


# Execução e comparação

N = 50_000_000   # 50 milhões

println("\n--- Executando em SÉRIE ---")
@time result_serie = loop_serie(N)

println("\n--- Executando em PARALELO ---")
@time result_paralelo = loop_paralelo(N)



Threads ativos = 2

--- Executando em SÉRIE ---
  2.748832 seconds (6.78 k allocations: 346.727 KiB, 0.64% compilation time)

--- Executando em PARALELO ---
  1.630133 seconds (58.61 k allocations: 2.983 MiB, 17.01% compilation time)


9.550964498552416e6

| Execução                     | Tempo (s)   | Alocações       | Memória usada | Resultado numérico        |
|------------------------------|-------------|-----------------|----------------|----------------------------|
| Sem Paralelismo (Série)      | 2.748832 s  | 6.78 mil        | 346.727 KiB    | —                          |
| Com Paralelismo (2 threads)  | 1.630133 s  | 58.61 mil       | 2.983 MiB      | 9.550964498552416e6        |


Nesse exemplo o loop em paralelo foi mais vantajoso pois:

1 - Com o número de interações extremamento grande, no paralelismo é dividido o trabalho entre vários threads, ajudando a reduzir diretamento o tempo total

2 - Como cada iteração era independete, ou seja, não depende da anterior, não gera conflitos de memória, permitindo que cada thread calcule seu valor separadamente, e quanto maior o threads maior o tempo ganho

3 - E o custo da paralelização foi menor que o ganho, pois aqui, diferente do exemplo anterior, o volume gigantesco de iterações superou o custo das threads

In [13]:
using Dates

# Data e hora completas
agora = now()

# Formatando de forma amigável para relatórios
timestamp = Dates.format(agora, "dd/MM/yyyy HH:mm:ss")

println("Data e hora da geração do relatório: ", timestamp)


Data e hora da geração do relatório: 18/16/2025 02:11:604
